In [30]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from nltk.tokenize.api import StringTokenizer

In [11]:
corpus = 'You will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. I arrived here yesterday, and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking.'

In [59]:
split_corpus = corpus.split(' ')
vocabulary = list(set(split_corpus)) # Token should only appear once
vocabulary_indices = list(range(len(vocabulary)))
vocabulary_size = len(vocabulary_indices)
word_to_index = dict(zip(vocabulary, vocabulary_indices))

In [21]:
word_to_index

{'the': 0,
 'dear': 1,
 'here': 2,
 'such': 3,
 'no': 4,
 'I': 5,
 'first': 6,
 'disaster': 7,
 'in': 8,
 'undertaking.': 9,
 'welfare': 10,
 'has': 11,
 'hear': 12,
 'which': 13,
 'that': 14,
 'my': 15,
 'an': 16,
 'task': 17,
 'commencement': 18,
 'arrived': 19,
 'increasing': 20,
 'you': 21,
 'is': 22,
 'confidence': 23,
 'accompanied': 24,
 'of': 25,
 'have': 26,
 'enterprise': 27,
 'You': 28,
 'to': 29,
 'evil': 30,
 'sister': 31,
 'rejoice': 32,
 'will': 33,
 'assure': 34,
 'with': 35,
 'forebodings.': 36,
 'yesterday,': 37,
 'and': 38,
 'regarded': 39,
 'success': 40}

In [34]:
WINDOW_SIZE = 2
corpus_size = len(split_corpus)


index_pairs = []
for position, word in enumerate(split_corpus):
    window_minimum = max(position - WINDOW_SIZE, 0)
    window_maximum = min(position + WINDOW_SIZE + 1, corpus_size)
    for window_position in range(window_minimum, window_maximum):
        if position != window_position:
            index_pairs.append((word_to_index[split_corpus[window_position]], word_to_index[word]))

index_pairs = np.array(index_pairs)

In [35]:
EMBEDDING_DIMENSIONS = 5
LEARNING_RATE = 0.001
NUMBER_OF_EPOCHES = 10

In [50]:
def one_hot_encoding(position):
    one_hot_vector = torch.zeros(vocabulary_size).float()
    one_hot_vector[position] = 1.0
    return one_hot_vector

In [64]:
InputEmbeddingLayer = Variable(torch.randn(EMBEDDING_DIMENSIONS, vocabulary_size).float(), requires_grad=True)
OutputEmbeddingLayer = Variable(torch.randn(vocabulary_size, EMBEDDING_DIMENSIONS).float(), requires_grad=True)

for epoch in range(NUMBER_OF_EPOCHES):
    epoch_loss = 0
    for context, target in index_pairs:
        input_vector = Variable(one_hot_encoding(context)).float()
        ground_truth = Variable(torch.from_numpy(np.array([target])).long())
        
        input_embedding = torch.matmul(InputEmbeddingLayer, input_vector)
        output_embedding = torch.matmul(OutputEmbeddingLayer, input_embedding)
        
        log_softmax = F.log_softmax(output_embedding, dim=0)
        loss = F.nll_loss(log_softmax.unsqueeze(0), ground_truth)
        epoch_loss += loss.item()
        
        loss.backward()
        
        InputEmbeddingLayer.data -= LEARNING_RATE * InputEmbeddingLayer.grad.data
        OutputEmbeddingLayer.data -= LEARNING_RATE * OutputEmbeddingLayer.grad.data
        
        InputEmbeddingLayer.grad.data.zero_()
        OutputEmbeddingLayer.grad.data.zero_()
    if epoch % 10 == 0:
        print(epoch_loss / len(index_pairs))
        

6.03733890558544
5.567180040008143
5.220810454770138
4.951218163339715
4.734942854078192
4.557548101324784
4.409391243834245
4.283826636013232
4.176110553741455
4.082727767291822
4.000993476415935
